[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)  
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from operator import itemgetter


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    # max_tokens=512,
)

## 6. trim 설정
trimmer = trim_messages(
    max_tokens=65,      ## 최대 토큰 수 제한
    strategy='last',    ## 최근 메시지부터 시작해서 자르기
    token_counter=llm,  ## 모델 기준으로 토큰 수 계산
    include_system=True,## 시스템 메시지 포함
    allow_partial=False, ## 메시지 자르기 제한
    start_on='human',   ## human 메시지부터 자르기 시작
)

## 7. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)

## 8. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 9. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )

    print('\n' + '=' * 50 + '\n')

In [11]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은 얼마?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간은 1년 이내로 할 수 있습니다. 다만, 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모, 「한부모가족지원법」 제4조제1호의 부 또는 모, 고용노동부령으로 정하는 장애아동의 부 또는 모인 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 제2항)', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-e7b9ad92-8883-4be1-b030-0dc61353a87b-0'), HumanMessage(content='육아휴직 대상자는?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 대상자는 임신 중인 여성 근로자와 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 사용자는 이들이 육아휴직을 신청하는 경우 이를 허용해야 합니다. 다만, 대통령령으로 정하는 예외적인 경우에는 허용하지 않을 수 있습니다 (제19조 제1항).', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-d5b0216e-bf31-4a84-bc4f-804f5c1aa725-0'), HumanMessage(content='육아휴직 수당은?'

In [12]:
store.keys()

dict_keys(['abc'])

In [13]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 육아휴직 기간은 얼마?

[AI]: 육아휴직의 기간은 1년 이내로 할 수 있습니다. 다만, 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모, 「한부모가족지원법」 제4조제1호의 부 또는 모, 고용노동부령으로 정하는 장애아동의 부 또는 모인 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 제2항)

[HUMAN]: 육아휴직 대상자는?

[AI]: 육아휴직의 대상자는 임신 중인 여성 근로자와 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 사용자는 이들이 육아휴직을 신청하는 경우 이를 허용해야 합니다. 다만, 대통령령으로 정하는 예외적인 경우에는 허용하지 않을 수 있습니다 (제19조 제1항).

[HUMAN]: 육아휴직 수당은?

[AI]: 육아휴직 수당과 관련하여, 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률에는 육아휴직 기간 동안의 생계비용과 관련된 직접적인 내용은 포함되어 있지 않습니다. 다만, 제20조에 따르면 국가는 사업주가 근로자에게 육아휴직이나 육아기 근로시간 단축을 허용한 경우 그 근로자의 생계비용과 사업주의 고용유지비용의 일부를 지원할 수 있다고 명시되어 있습니다.

더 구체적인 육아휴직 수당 지급 기준이나 방법에 대해서는 관련 법률이나 지침에서 다루어질 수 있으며, 추가로 고용보험 등 다른 제도를 통해 지원받을 수 있습니다. 육아휴직 수당에 대한 상세한 정보는 고용보험 관련 법규나 지침을 참고하시기 바랍니다.

